# Use BoTorch for feasibility weighted acquisition function

import libearies

In [13]:
import pandas as pd
import numpy as np
import torch
import botorch
import gpytorch
import matplotlib.pyplot as plt

#modules for regression
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize

#modules for BO
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.utils.sampling import sample_simplex
from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.dominated import (
    DominatedPartitioning,
)
from botorch.utils.multi_objective.pareto import is_non_dominated

from gpytorch.likelihoods import DirichletClassificationLikelihood


Read and normalize data

In [2]:
filename = r'../data/olhs_run1.xlsx'
x_pd = pd.read_excel(filename, sheet_name='Initial Design (OLHS)', header=[0,1], index_col=[0])
y_pd = pd.read_excel(filename, sheet_name='bo_data', header=[0,1], index_col=[0])

dtype=torch.double

objective_properties = ['Polymer Solubility', 'Gelation Enthalpy', 'Shear Modulus']

x = torch.tensor(x_pd.values, dtype=dtype)
y = torch.tensor(y_pd[objective_properties].values, dtype=dtype)
mfg = torch.tensor(y_pd['Manufacturability'].values, dtype=torch.long)

x_bounds = np.array([[2000, 10000], [0, 100], [0, 40], [5000, 15000], [80, 100], [0,100], [60, 100], [70, 100]])
x_bounds = torch.tensor(x_bounds.T, dtype=dtype)

x = normalize(x, bounds=x_bounds)

Helper functions

Define and initialize regression and classification model

In [14]:
# Regression model
models = []
for i in range(len(objective_properties)):
    models.append(SingleTaskGP(x, y[:, i].unsqueeze(-1), 
                               outcome_transform=Standardize(m=1)))

# transformation of mfg labels
tmp_likl = DirichletClassificationLikelihood(targets=mfg.squeeze(), alpha_epsilon=1e-4, 
                                             learn_additional_noise=False)
cls_model = SingleTaskGP(train_X=x, train_Y=tmp_likl.transformed_targets.T.double(), 
                         outcome_transform=Standardize(m=2))

models.append(cls_model)

model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)
mll = mll.to(x)

Fit the models to data

In [15]:
# regression model
fit_gpytorch_mll(mll)

SumMarginalLogLikelihood(
  (likelihood): LikelihoodList(
    (likelihoods): ModuleList(
      (0-3): 4 x GaussianLikelihood(
        (noise_covar): HomoskedasticNoise(
          (noise_prior): GammaPrior()
          (raw_noise_constraint): GreaterThan(1.000E-04)
        )
      )
    )
  )
  (model): ModelListGP(
    (models): ModuleList(
      (0-3): 4 x SingleTaskGP(
        (likelihood): GaussianLikelihood(
          (noise_covar): HomoskedasticNoise(
            (noise_prior): GammaPrior()
            (raw_noise_constraint): GreaterThan(1.000E-04)
          )
        )
        (mean_module): ConstantMean()
        (covar_module): ScaleKernel(
          (base_kernel): MaternKernel(
            (lengthscale_prior): GammaPrior()
            (raw_lengthscale_constraint): Positive()
          )
          (outputscale_prior): GammaPrior()
          (raw_outputscale_constraint): Positive()
        )
        (outcome_transform): Standardize()
      )
    )
    (likelihood): LikelihoodList

Prediction for sanity check

In [16]:
model.eval()

with torch.no_grad():
    posterior = model.posterior(x)
    pred_mean = posterior.mean
pred_mean

tensor([[ 53.0025,   4.3507,   5.1600, -13.3763,  -0.7857],
        [ 23.5312,  54.3269,   1.0192,  -1.2660, -12.8960],
        [ 38.3127,   6.4286,   1.5165,  -1.2760, -12.8860],
        [ 31.9368,   5.7843,   1.4786,  -1.2722, -12.8898],
        [120.8128,   3.3749,   2.3164, -13.4369,  -0.7251],
        [ 81.0235,  83.1740,   1.5724, -13.4353,  -0.7267],
        [ 84.5024,   3.5945,   2.7505, -13.3966,  -0.7654],
        [ 65.4618,  28.0365,   3.4917, -13.3777,  -0.7843],
        [ 49.5438, 107.8939,   5.2615, -13.3980,  -0.7640],
        [167.9699,   4.9255,   9.9088, -13.4172,  -0.7448],
        [ 49.5146, 113.7600,   0.7719, -13.3916,  -0.7704],
        [102.1682,  18.2934,   2.3548, -13.4071,  -0.7549],
        [ 81.0200,  10.2873,   9.3325, -13.4192,  -0.7428],
        [ 65.0121, 105.9470,  39.1052, -13.4326,  -0.7294],
        [ 37.5390,  44.1415,   1.3258,  -1.2345, -12.9275],
        [117.9476,   4.5463,   9.5441, -13.4365,  -0.7255],
        [148.2253,  17.0707,  27.8040, -

Helper functions for BO

In [10]:
bounds = torch.zeros(2, 8)
bounds[1] = 1

BATCH_SIZE = 4      # Number of candidates selected in each BO run/iteration
NUM_RESTARTS = 10   # Restarts during BO run
RAW_SAMPLES = 512   


In [ ]:
def constraint_callable(Z):
    class0 = Z[..., 3]
    class1 = Z[..., 4]
    return class0 - class1